In [7]:
!git clone https://github.com/google-research/timesfm.git

Cloning into 'timesfm'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (651/651), done.
remote: Compressing objects: 100% (326/326), done.
remote: Total 653 (delta 344), reused 541 (delta 284), pack-reused 2 (from 1)
Receiving objects: 100% (653/653), 1.91 MiB | 8.56 MiB/s, done.
Resolving deltas: 100% (344/344), done.


In [8]:
!pip install yfinance timesfm==1.1.0

INFO: pip is looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warning

In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3fa5c866d783e5d433422651ccc76b832a848419df7cba09649e00dbec11bc1d
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [1]:
import os
import time
import timesfm
from collections import defaultdict
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error


2024-11-08 13:57:10.659765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Load TimesFM model
timesfm_backend = "gpu"  # or 'cpu'
from jax._src import config
config.update("jax_platforms", {"cpu": "cpu", "gpu": "cuda", "tpu": ""}[timesfm_backend])

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# انتخاب نماد سهام (مثال: AAPL برای اپل)
ticker = 'AAPL'

# بارگذاری داده‌های بورسی روزانه
data = yf.download(ticker, start='1900-01-01', end='2024-01-01')
data = data.dropna()
data.reset_index(inplace=True)  # تبدیل شاخص به ستون

# تغییر نام ستون‌ها
data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)



# محاسبه ویژگی‌های اضافی
data['MA_10'] = data['y'].rolling(window=10).mean()
data['MA_50'] = data['y'].rolling(window=50).mean()
data['RSI'] = 100 - (100 / (1 + data['y'].pct_change().rolling(window=14).apply(lambda x: (x[x > 0].mean() / -x[x < 0].mean()) if x[x < 0].mean() != 0 else np.inf)))



# انتخاب ستون‌های نهایی
final_cols = ["date",  "y"]
data = data[final_cols]
data.columns = data.columns.droplevel(1)

# ذخیره‌سازی داده‌ها در یک فایل CSV
data_path = 'stock_data.csv'
data.to_csv(data_path, index=False)

print(f"Number of records: {len(data)}")


[*********************100%***********************]  1 of 1 completed


Number of records: 10853


In [5]:
data

Price,date,y
0,1980-12-12 00:00:00+00:00,0.128348
1,1980-12-15 00:00:00+00:00,0.121652
2,1980-12-16 00:00:00+00:00,0.112723
3,1980-12-17 00:00:00+00:00,0.115513
4,1980-12-18 00:00:00+00:00,0.118862
...,...,...
10848,2023-12-22 00:00:00+00:00,193.600006
10849,2023-12-26 00:00:00+00:00,193.050003
10850,2023-12-27 00:00:00+00:00,193.149994
10851,2023-12-28 00:00:00+00:00,193.580002


In [4]:
import sys
import os

# اضافه کردن مسیر ماژول به sys.path
sys.path.append(os.path.join(os.getcwd(), 'timesfm/src/adapter'))

# حالا می‌توانید ماژول‌ها را وارد کنید
from utils import get_adapter_params, load_adapter_layer

In [5]:
import gc
import logging
import warnings
from datetime import datetime
from typing import Tuple

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
from jax import numpy as jnp
from paxml import checkpoint_types, checkpoints, learners, tasks_lib, trainer_lib
from praxis import optimizers, pax_fiddle, py_utils, schedules
from tqdm import tqdm
from typing_extensions import Annotated
from timesfm import TimesFm, data_loader, patched_decoder

NestedMap = py_utils.NestedMap

warnings.filterwarnings("ignore")
cmdstanpy_logger = logging.getLogger("cmdstanpy")
absl_logger = logging.getLogger("absl")
cmdstanpy_logger.disabled = True
absl_logger.disabled = True

INPUT_PATCH_LEN = 32
OUTPUT_PATCH_LEN = 128
NUM_LAYERS = 20
MODEL_DIMS = 1280
QUANTILES = list(np.arange(1, 10) / 10.0)
EPS = 1e-7
RANDOM_SEED = 1234

def finetune(
    model_name: str = "google/timesfm-1.0-200m",
    checkpoint_path: str = None,
    datetime_col: str = "date",
    ts_cols: list[str] = [],
    normalize: bool = True,
    num_cov_cols: int = None,
    context_len: int = 64,
    horizon_len: int = 1,
    freq: str = "D",
    data_path: str = "timeseries_data.csv",
    boundaries: Tuple[int, int, int] = (0, 0, 0),
    backend: str = "gpu",
    batch_size: int = 32,
    num_epochs: int = 10,
    learning_rate: float = 0.001,
    adam_epsilon: float = 1e-8,
    adam_clip_threshold: float = 1.0,
    cos_initial_decay_value: float = 0.001,
    cos_final_decay_value: float = 0.0001,
    cos_decay_steps: int = 1000,
    ema_decay: float = 0.9999,
    early_stop_patience: int = 5,
    use_lora: bool = False,
    lora_rank: int = 8,
    lora_target_modules: str = "all",
    use_dora: bool = False,
    use_linear_probing: bool = False,
    checkpoint_dir: str = "./checkpoints",
) -> None:
    key = jax.random.PRNGKey(seed=RANDOM_SEED)

    data_df = pd.read_csv(open(data_path, "r"))



    ts_cols = [col for col in data_df.columns if col != datetime_col]

    dtl = data_loader.TimeSeriesdata(
        data_path=data_path,
        datetime_col=datetime_col,
        num_cov_cols=num_cov_cols,
        cat_cov_cols=None,
        ts_cols=np.array(ts_cols),
        train_range=[0, boundaries[0]],
        val_range=[boundaries[0], boundaries[1]],
        test_range=[boundaries[1], boundaries[2]],
        hist_len=context_len,
        pred_len=horizon_len,
        batch_size=batch_size,
        freq=freq,
        normalize=normalize,
        epoch_len=None,
        holiday=False,
        permute=False,
    )

    train_batches = dtl.tf_dataset(mode="train", shift=1).batch(batch_size)
    val_batches = dtl.tf_dataset(mode="val", shift=horizon_len)
    test_batches = dtl.tf_dataset(mode="test", shift=horizon_len)
    mse_losses = []

    for tbatch in tqdm(train_batches.as_numpy_iterator()):
        pass
    print(tbatch[0].shape)

    tfm = TimesFm(
        context_len=context_len,
        horizon_len=horizon_len,
        input_patch_len=INPUT_PATCH_LEN,
        output_patch_len=OUTPUT_PATCH_LEN,
        num_layers=NUM_LAYERS,
        model_dims=MODEL_DIMS,
        backend=backend,
        per_core_batch_size=batch_size,
        quantiles=QUANTILES,
    )


    if checkpoint_path:
        tfm.load_from_checkpoint(
            checkpoint_path=checkpoint_path,
            checkpoint_type=checkpoints.CheckpointType.FLAX,
        )
    else:
        tfm.load_from_checkpoint(
            repo_id=model_name,
            checkpoint_type=checkpoints.CheckpointType.FLAX,
        )

    model = pax_fiddle.Config(
        patched_decoder.PatchedDecoderFinetuneModel,
        name="patched_decoder_finetune",
        core_layer_tpl=tfm.model_p,
    )
    for batch in tqdm(test_batches.as_numpy_iterator()):
        past = batch[0]
        actuals = batch[3]

        _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
        forecasts = forecasts[:, 0 : actuals.shape[1], 5]

        mse_losses.append(np.square(forecasts - actuals).mean())

    print(f"MSE: {np.mean(mse_losses)}")

    if use_lora:
        load_adapter_layer(
            mdl_vars=tfm._train_state.mdl_vars,
            model=model.core_layer_tpl,
            lora_rank=lora_rank,
            lora_target_modules=lora_target_modules,
            use_dora=use_dora,
        )

    @pax_fiddle.auto_config
    def build_learner() -> learners.Learner:
        bprop_variable_inclusion = []
        bprop_variable_exclusion = []
        if use_lora:
            bprop_variable_inclusion.append(r"^.*lora.*$")
            if use_dora:
                bprop_variable_inclusion.append(r"^.*dora.*$")
        elif use_linear_probing:
            bprop_variable_exclusion = [".*/stacked_transformer_layer/.*"]

        return pax_fiddle.Config(
            learners.Learner,
            name="learner",
            loss_name="avg_qloss",
            optimizer=optimizers.Adam(
                epsilon=adam_epsilon,
                clip_threshold=adam_clip_threshold,
                learning_rate=learning_rate,
                lr_schedule=pax_fiddle.Config(
                    schedules.Cosine,
                    initial_value=cos_initial_decay_value,
                    final_value=cos_final_decay_value,
                    total_steps=cos_decay_steps,
                ),
                ema_decay=ema_decay,
            ),
            bprop_variable_exclusion=bprop_variable_exclusion,
            bprop_variable_inclusion=bprop_variable_inclusion,
        )

    task_p = tasks_lib.SingleTask(
        name="ts-learn",
        model=model,
        train=tasks_lib.SingleTask.Train(
            learner=build_learner(),
        ),
    )

    task_p.model.ici_mesh_shape = [1, 1, 1]
    task_p.model.mesh_axis_names = ["replica", "data", "mdl"]

    DEVICES = np.array(jax.devices()).reshape([1, 1, 1])
    jax.sharding.Mesh(DEVICES, ["replica", "data", "mdl"])

    num_devices = jax.local_device_count()
    print(f"num_devices: {num_devices}")
    print(f"device kind: {jax.local_devices()[0].device_kind}")

    jax_task = task_p
    key, init_key = jax.random.split(key)

    def process_train_batch(batch):

        past_ts = batch[0].reshape( batch[0].shape[0] * len(ts_cols), -1)
        actual_ts = batch[3].reshape(batch[3].shape[0] * len(ts_cols), -1)
        return NestedMap(input_ts=past_ts, actual_ts=actual_ts)

    def process_eval_batch(batch):
        past_ts = batch[0]
        actual_ts = batch[3]
        return NestedMap(input_ts=past_ts, actual_ts=actual_ts)

    jax_model_states, _ = trainer_lib.initialize_model_state(
        jax_task,
        init_key,
        process_train_batch(tbatch),
        checkpoint_type=checkpoint_types.CheckpointType.GDA,
    )
    jax_model_states.mdl_vars["params"]["core_layer"] = tfm._train_state.mdl_vars[
        "params"
    ]
    gc.collect()

    jax_task = task_p

    def train_step(states, prng_key, inputs):
        return trainer_lib.train_step_single_learner(jax_task, states, prng_key, inputs)

    def eval_step(states, prng_key, inputs):
        states = states.to_eval_state()
        return trainer_lib.eval_step_single_learner(jax_task, states, prng_key, inputs)

    key, train_key, eval_key = jax.random.split(key, 3)
    train_prng_seed = jax.random.split(train_key, num=jax.local_device_count())
    eval_prng_seed = jax.random.split(eval_key, num=jax.local_device_count())

    p_train_step = jax.pmap(train_step, axis_name="batch")
    p_eval_step = jax.pmap(eval_step, axis_name="batch")

    replicated_jax_states = trainer_lib.replicate_model_state(jax_model_states)

    def reshape_batch_for_pmap(batch, num_devices):
        def _reshape(input_tensor):
            bsize = input_tensor.shape[0]
            residual_shape = list(input_tensor.shape[1:])
            nbsize = bsize // num_devices
            return jnp.reshape(input_tensor, [num_devices, nbsize] + residual_shape)

        return jax.tree.map(_reshape, batch)

    patience = 0
    best_eval_loss = 1e7
    checkpoint_dir = f"{checkpoint_dir}/run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    for epoch in range(num_epochs):
        if patience >= early_stop_patience:
            print("Early stopping.")
            break
        print(f"Epoch: {epoch + 1}")
        train_its = train_batches.as_numpy_iterator()
        train_losses = []
        for batch in tqdm(train_its):
            tbatch = process_train_batch(batch)
            tbatch = reshape_batch_for_pmap(tbatch, num_devices)
            replicated_jax_states, step_fun_out = p_train_step(
                replicated_jax_states, train_prng_seed, tbatch
            )
            train_losses.append(step_fun_out.loss[0])

        avg_train_loss = np.mean(train_losses)

        print("Starting eval.")
        val_its = val_batches.as_numpy_iterator()
        eval_losses = []
        for ev_batch in tqdm(val_its):
            ebatch = process_eval_batch(ev_batch)
            ebatch = reshape_batch_for_pmap(ebatch, num_devices)
            _, step_fun_out = p_eval_step(replicated_jax_states, eval_prng_seed, ebatch)
            eval_losses.append(step_fun_out.loss[0])

        avg_eval_loss = np.mean(eval_losses)

        print(f"Train Loss: {avg_train_loss}, Val Loss: {avg_eval_loss}")

        if avg_eval_loss < best_eval_loss or np.isnan(avg_eval_loss):
            best_eval_loss = avg_eval_loss
            print("Saving checkpoint.")
            jax_state_for_saving = py_utils.maybe_unreplicate_for_fully_replicated(
                replicated_jax_states
            )
            if use_lora:
                adapter_params = get_adapter_params(
                    params=jax_state_for_saving.mdl_vars,
                    lora_target_modules=lora_target_modules,
                    num_layers=NUM_LAYERS,
                    use_dora=use_dora,
                )
                jax_state_for_saving.mdl_vars["params"] = adapter_params

            checkpoints.save_checkpoint(
                jax_state_for_saving, checkpoint_dir, overwrite=True
            )

            patience = 0
            del jax_state_for_saving
            gc.collect()
        else:
            patience += 1
            print(f"patience: {patience}")
    print("Fine-tuning completed.")
    return (dtl,test_batches)


In [6]:
ts_cols = [col for col in data.columns if col != "date"]
train_data = data[data['date'] < '2022-07-01']
val_data = data[(data['date'] >= '2022-07-01') & (data['date'] < '2023-01-01')]
test_data = data[data['date'] >= '2023-01-01']
dtl,test_batches=finetune(
    model_name="google/timesfm-1.0-200m",  # یا هر مدل دیگر
    checkpoint_path=None,  # اگر از چک‌پوینت خاصی می‌خواهید استفاده کنید، مسیر را مشخص کنید
    datetime_col="date",
    ts_cols=["y"],
    num_cov_cols=None,  # تعداد ستون‌های کوواریانس عددی
    context_len=64,  # طول پنجره زمینه
    horizon_len=1,  # طول پیش‌بینی
    freq="D",  # فرکانس داده‌ها (روزانه)
    data_path="stock_data.csv",  # مسیر فایل CSV
    backend="gpu",  # یا 'cpu' یا 'tpu'
    batch_size=32,
    num_epochs=100,
    use_lora=True,
    learning_rate=0.001,
    adam_epsilon=1e-8,
    adam_clip_threshold=1.0,
    boundaries= (len(train_data)+64, len(train_data)+len(val_data)+64, len(data)),
    cos_initial_decay_value=0.001,
    cos_final_decay_value=0.0001,
    cos_decay_steps=1000,
    ema_decay=0.9999,
    early_stop_patience=5,
    checkpoint_dir="./checkpoints",

)



2024-11-08 13:57:57.548151: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.77). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2024-11-08 13:57:58.182582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-11-08 13:57:58.182918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-11-08 13:57:58.183073: W tensorflow/stream_executor/platform/defa

(11, 1, 64)
Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.
Constructed model weights in 4.01 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.
Restored checkpoint in 4.74 seconds.
Jitting decoding.
Jitted decoding in 25.16 seconds.


186it [00:03, 57.00it/s]


...

Train Loss: 0.04369119182229042, Val Loss: 0.5199756026268005
patience: 4
Epoch: 29


328it [00:33,  9.67it/s]


Starting eval.


127it [00:03, 41.61it/s]


Train Loss: 0.04360850155353546, Val Loss: 0.5199511647224426
patience: 5
Early stopping.
Fine-tuning completed.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from timesfm import TimesFm, freq_map, data_loader

from tqdm import tqdm
import numpy as np
import pandas as pd


tfm = TimesFm(
    context_len=64,
    horizon_len=128,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.
Constructed model weights in 3.84 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.
Restored checkpoint in 3.93 seconds.
Jitting decoding.
Jitted decoding in 33.44 seconds.


In [10]:
import sys
import os

# اضافه کردن مسیر ماژول به sys.path
sys.path.append(os.path.join(os.getcwd(), 'timesfm/src/adapter'))

# حالا می‌توانید ماژول‌ها را وارد کنید
from utils import load_adapter_checkpoint, load_adapter_layer

In [11]:
load_adapter_checkpoint(
    model=tfm,
    adapter_checkpoint_path="/content/checkpoints/run_20241108_135904",
    lora_rank=8,
    lora_target_modules="all",
    use_dora=False,

)

Restoring adapter checkpoint from /content/checkpoints/run_20241108_135904.
Restored adapter checkpoint in 9.75 seconds.
Jitting decoding.
Jitted decoding in 21.15 seconds.


In [14]:
    mse_losses = []
    for batch in tqdm(test_batches.as_numpy_iterator()):
        past = batch[0]
        actuals = batch[3]

        _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
        forecasts = forecasts[:, 0 : actuals.shape[1], 5]

        mse_losses.append(np.square(forecasts - actuals).mean())

    print(f"MSE: {np.mean(mse_losses)}")

186it [00:03, 55.08it/s]

MSE: 0.004359005484730005


In [15]:
from collections import defaultdict
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")
data = data.dropna()

df = data[['Close']].reset_index()
df.columns = ['date', 'close']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn
context_len=64
input_patch_len=32
horizon_len=1
batch_size=1

input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)

results = []
# Process in smaller sub-batches
for i, example in enumerate(input_data()):
    raw_forecast, _ = tfm.forecast(
        inputs=example["inputs"], freq=[0] * len(example["inputs"])
    )
    for j in range(len(example["inputs"])):
        result = {
            'input_index': j,
            'train_start_date': str(example["dates"][j][0]),
            'train_end_date': str(example["dates"][j][-2]),
            'predict_date': str(example["dates"][j][-1]),
            'raw_forecast': raw_forecast[j][0],
            'y_actual': example["outputs"][j][0],
            'batch_size': batch_size,
            'context_len': context_len,
            'input_patch_len': input_patch_len
        }
        results.append(result)
results_df = pd.DataFrame(results)
from sklearn.metrics import mean_squared_error, accuracy_score
mean_squared_error(results_df['y_actual'], results_df['raw_forecast'])


[*********************100%***********************]  1 of 1 completed


5.03608915312845